# Retrieval Augumented Generation (RAG) inference

***This notebook works best with the `conda_python3` on the `ml.t3.large` instance***.

---

At this point our slide deck data is ingested into Amazon OpenSearch Service Serverless collection. We are now ready to talk to our slide deck using a large multimodal model. We are using the [Anthropic’s Claude 3 Sonnet foundation model](https://aws.amazon.com/about-aws/whats-new/2024/03/anthropics-claude-3-sonnet-model-amazon-bedrock/) for this purpose.

## Step 1. Setup

Install the required Python packages and import the relevant files.

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

In [1]:
import os
import io
import sys
import json
import glob
import boto3
import codecs
import base64
import logging
import requests
import botocore
import sagemaker
import jsonlines
import numpy as np
import pandas as pd
import globals as g
from PIL import Image
from pathlib import Path
from typing import List, Dict
from IPython.display import Image
from urllib.parse import urlparse
from botocore.auth import SigV4Auth
from pandas.core.series import Series
from sagemaker import get_execution_role
from botocore.awsrequest import AWSRequest
from utils import get_cfn_outputs, get_text_embedding, get_llm_response, find_similar_data
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

## Step 2. Create an OpenSearch client and SageMaker Predictor object

We create an OpenSearch client so that we can query the vector database for embeddings (slides) similar to the questions that we might want to ask of our slide deck and then we create a SageMaker [`Predictor`](https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html) to run inference using the LLaVA model given the slide we retrieved from OpenSearch.

Get the name of the OpenSearch Service Serverless collection endpoint and index name from the CloudFormation stack outputs.

In [3]:
outputs = get_cfn_outputs(g.CFN_STACK_NAME)
host = outputs['MultimodalCollectionEndpoint'].split('//')[1]
# index_name = outputs['OpenSearchIndexName']
index_name = "blog3slides-app2"
logger.info(f"opensearchhost={host}, index={index_name}")


[2024-05-29 04:09:55,819] p24700 {3133969832.py:5} INFO - opensearchhost=7uiiz7d87b3q8u2kfmtd.us-east-1.aoss.amazonaws.com, index=blog3slides-app2


In [4]:
session = boto3.Session()
credentials = session.get_credentials()
auth = AWSV4SignerAuth(credentials, g.AWS_REGION, g.OS_SERVICE)

os_client = OpenSearch(
    hosts = [{'host': host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    pool_maxsize = 20
)

[2024-05-29 04:09:56,627] p24700 {credentials.py:1075} INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


## Step 3. Read for RAG

We now have all the pieces for RAG. Here is how we _talk to our slide deck_.

1. Convert the user question into embeddings using the Titan Text Embeddings model.

1. Find the most similar slide (image) corresponding to the the embeddings (for the user question) from the vector database (OpenSearch Serverless).

1. Now ask Claude3 to answer the user question using the retrieved image description for the most similar slide.

In [5]:
bedrock = boto3.client(service_name="bedrock-runtime", endpoint_url=g.TITAN_URL)

A handy function for similarity search in the vector db

### Ask questions
Loop through questions in the jsonl file to -
1. Embed the question
2. Do a similarity search to retrive the closest image url and image description
3. Get final response from Claude by passing question and image description
4. Append responses to a list

Save the responses

In [8]:
responses_list = []
with jsonlines.open('qa.jsonl') as f:
    for line in f.iter():
        question: str = line['question']
        text_embedding = get_text_embedding(bedrock, question, g.TITAN_MODEL_ID)
        vector_db_response: Dict = find_similar_data(os_client, text_embedding, 1, index_name)
        deck_name = vector_db_response.get('hits', {}).get('hits')[0].get('_source').get('metadata').get('deck_name')
        deck_url = vector_db_response.get('hits', {}).get('hits')[0].get('_source').get('metadata').get('deck_url')
        slide_text = vector_db_response.get('hits', {}).get('hits')[0].get('_source').get('slide_text')
        img_url = vector_db_response.get('hits', {}).get('hits')[0].get('_source').get('image_url')
        
        logger.info(f"going to answer the question=\"{question}\" using the image \"{img_url}\"")
        llm_response = get_llm_response(bedrock, question, slide_text)

        response = {
            "question": question,
            "response": {
                "resp_txt": llm_response,
                "resp_img_url": img_url,
                "resp_deck_name": deck_name,
                "resp_deck_url": deck_url
            }
        }
        responses_list.append(response)
        logger.info(f"appended response corresponding to {question}")

[2024-05-29 04:10:43,174] p24700 {base.py:259} INFO - POST https://7uiiz7d87b3q8u2kfmtd.us-east-1.aoss.amazonaws.com:443/blog3slides-app2/_search [status:200 request:0.050s]
[2024-05-29 04:10:43,176] p24700 {utils.py:93} INFO - received response from OpenSearch
[2024-05-29 04:10:43,177] p24700 {1563184157.py:12} INFO - going to answer the question="In which year did Nestlé achieve higher Organic Growth, 2003 or 2004?" using the image "https://image.slidesharecdn.com/2012-02-20fy11roadshow-120221022442-phpapp02/95/feb-20-2012-nestl-2011-fullyear-roadshow-presentation-7-1024.jpg"
[2024-05-29 04:10:45,131] p24700 {1563184157.py:25} INFO - appended response corresponding to In which year did Nestlé achieve higher Organic Growth, 2003 or 2004?
[2024-05-29 04:10:45,398] p24700 {base.py:259} INFO - POST https://7uiiz7d87b3q8u2kfmtd.us-east-1.aoss.amazonaws.com:443/blog3slides-app2/_search [status:200 request:0.037s]
[2024-05-29 04:10:45,399] p24700 {utils.py:93} INFO - received response from 

In [9]:
fpath: str = "responses-appr2.json"
json.dump(responses_list, codecs.open(fpath, 'w', encoding='utf-8'), 
          separators=(',', ':'), 
          sort_keys=True, 
          indent=4)
logger.info(f"saved responses for all questions in {fpath}")

[2024-05-29 04:36:58,959] p24700 {1177031128.py:6} INFO - saved responses for all questions in responses-appr2.json
